# 核心能力提升班商业智能方向 004期 Week 9

### Thinking 1: 常用的文本分类方法都有哪些

1. 朴素贝叶斯分类（Naïve Bayes）：朴素贝叶斯分类器基于属性条件独立假设：对于已知类别，假设所有属性相互独立，也就是说，假设每个属性独立地对分类结果产生影响。
2. KNN文本分类算法：KNN法由Cover和Hart于1968年提出，是一个理论上比较成熟的方法。该算法的基本思想是:根据传统的向量空间模型，文本内容被形式化为特征空间中的加权特征向量。对于一个测试文本，计算它与训练样本集中每个文本的相似度，找出K个最相似的文本，根据加权距离和判断测试文本所属的类别，具体算法步骤如下:
   a)对于一个测试文本，根据特征词形成测试文本向量。
   b)计算该测试文本与训练集中每个文本的文本相似度，按照文本相似度，在训练文本集中选出与测试文本最相似的k个文本。
   c)在测试文本的k个近邻中，依次计算每类的权重。
   d)比较类的权重，将文本分到权重最大的那个类别中。
3. 支持向量机（SVM）算法：是一种建立在统计学习理论基础上的机器学习方法。该算法基于结构风险最小化原理，将数据集合压缩到支持向量集合，学习得到分类决策函数。
4. fastText：其核心思想是将整篇文档的词及n-gram向量叠加平均得到文档向量，然后使用文档向量做softmax多分类。
5. TextCNN：在文本分类任务中可以利用CNN来提取句子中类似 n-gram 的关键信息。
6. TextRNN: 递归神经网络（RNN, Recurrent Neural Network），能够更好的表达上下文信息。具体在文本分类任务中，Bi-directional RNN（实际使用的是双向LSTM）从某种意义上可以理解为可以捕获变长且双向的的 "n-gram" 信息。
7. Bert: 通过Transformer捕捉语句中的双向关系。使用了Mask Language Model(MLM)和 Next Sentence Prediction(NSP) 的多任务预训练模型。使用海量语料数据进行预训练。

### Thinking 2: RNN为什么会出现梯度消失

看到网上一篇文章的解释很好，所以摘抄了下来：https://www.cnblogs.com/jins-note/p/10853788.html

经典的RNN结构如下图所示：  
<img src="./RNN.png"></img>  
假设我们的时间序列只有三段， $S_{0}$ 为给定值，神经元没有激活函数，则RNN最简单的前向传播过程如下：
$$S_{1}=W_{x}X_{1}+W_{s}S_{0}+b_{1}O_{1}=W_{o}S_{1}+b_{2}$$
$$S_{2}=W_{x}X_{2}+W_{s}S_{1}+b_{1}O_{2}=W_{o}S_{2}+b_{2}$$
$$S_{3}=W_{x}X_{3}+W_{s}S_{2}+b_{1}O_{3}=W_{o}S_{3}+b_{2}$$
假设在t=3时刻，损失函数为 $L_{3}=\frac{1}{2}(Y_{3}-O_{3})^{2} $。

则对于一次训练任务的损失函数为 $L=\sum_{t=0}^{T}{L_{t}} $，即每一时刻损失值的累加。

使用随机梯度下降法训练RNN其实就是对 $W_{x} $、$ W_{s}$ 、 $W_{o} $以及$ b_{1}b_{2}$ 求偏导，并不断调整它们以使L尽可能达到最小的过程。

现在假设我们我们的时间序列只有三段，$t1$，$t2$，$t3$。

我们只对t3时刻的 $W_{x}$、$W_{s}$、$W_{0}$ 求偏导（其他时刻类似）：

$$\frac{\partial{L_{3}}}{\partial{W_{0}}}=\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{W_{o}}}$$

$$\frac{\partial{L_{3}}}{\partial{W_{x}}}=\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{3}}}{\partial{W_{x}}}+\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{3}}}{\partial{S_{2}}}\frac{\partial{S_{2}}}{\partial{W_{x}}}+\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{3}}}{\partial{S_{2}}}\frac{\partial{S_{2}}}{\partial{S_{1}}}\frac{\partial{S_{1}}}{\partial{W_{x}}}$$

$$\frac{\partial{L_{3}}}{\partial{W_{s}}}=\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{3}}}{\partial{W_{s}}}+\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{3}}}{\partial{S_{2}}}\frac{\partial{S_{2}}}{\partial{W_{s}}}+\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{3}}}{\partial{S_{2}}}\frac{\partial{S_{2}}}{\partial{S_{1}}}\frac{\partial{S_{1}}}{\partial{W_{s}}}$$

可以看出对于 $W_{0}$ 求偏导并没有长期依赖，但是对于 $W_{x}$、$W_{s}$ 求偏导，会随着时间序列产生长期依赖。因为 $S_{t}$ 随着时间序列向前传播，而 $S_{t}$ 又是 $W_{x}$、$W_{s}$的函数。

根据上述求偏导的过程，我们可以得出任意时刻对 $W_{x}$、$W_{s}$ 求偏导的公式：

$$\frac{\partial{L_{t}}}{\partial{W_{x}}}=\sum_{k=0}^{t}{\frac{\partial{L_{t}}}{\partial{O_{t}}}\frac{\partial{O_{t}}}{\partial{S_{t}}}}(\prod_{j=k+1}^{t}{\frac{\partial{S_{j}}}{\partial{S_{j-1}}}})\frac{\partial{S_{k}}}{\partial{W_{x}}}$$

任意时刻对$W_{s}$ 求偏导的公式同上。

如果加上激活函数， $S_{j}=tanh(W_{x}X_{j}+W_{s}S_{j-1}+b_{1})$ ，

则 $\prod_{j=k+1}^{t}{\frac{\partial{S_{j}}}{\partial{S_{j-1}}}} = \prod_{j=k+1}^{t}{tanh^{'}}W_{s}$
激活函数tanh和它的导数图像如下。  
<img src="./tanh.png"></img>  
由上图可以看出 $tanh^{'}\leq1 $，对于训练过程大部分情况下$tanh$的导数是小于1的，因为很少情况下会出现$W_{x}X_{j}+W_{s}S_{j-1}+b_{1}=0$ ，如果 $W_{s}$ 也是一个大于0小于1的值，则当t很大时 $\prod_{j=k+1}^{t}{tanh^{'}}W_{s} $，就会趋近于0，和 $0.01^{50}$ 趋近与0是一个道理。同理当 $W_{s} $很大时$ \prod_{j=k+1}^{t}{tanh^{'}}W_{s} $就会趋近于无穷，这就是RNN中梯度消失和爆炸的原因。

至于怎么避免这种现象，再看看 $\frac{\partial{L_{t}}}{\partial{W_{x}}}=\sum_{k=0}^{t}{\frac{\partial{L_{t}}}{\partial{O_{t}}}\frac{\partial{O_{t}}}{\partial{S_{t}}}}(\prod_{j=k+1}^{t}{\frac{\partial{S_{j}}}{\partial{S_{j-1}}}})\frac{\partial{S_{k}}}{\partial{W_{x}}} $梯度消失和爆炸的根本原因就是 $\prod_{j=k+1}^{t}{\frac{\partial{S_{j}}}{\partial{S_{j-1}}}}$ 这一坨，要消除这种情况就需要把这一坨在求偏导的过程中去掉，至于怎么去掉，一种办法就是使$ {\frac{\partial{S_{j}}}{\partial{S_{j-1}}}}\approx1 $另一种办法就是使 ${\frac{\partial{S_{j}}}{\partial{S_{j-1}}}}\approx0 $。其实这就是LSTM做的事情。

#### LSTM如何解决梯度消失问题

先上一张LSTM的经典图：  
<img src="./lstm.png"></img>  
RNN梯度消失和爆炸的原因这篇文章中提到的RNN结构可以抽象成下面这幅图：  
<img src="./rnn_reson.png"></img>   
而LSTM可以抽象成这样：   
<img src="./lstm_reson.png"></img>    
三个×分别代表的就是forget gate，input gate，output gate，而我认为LSTM最关键的就是forget gate这个部件。这三个gate是如何控制流入流出的呢，其实就是通过下面 $f_{t},i_{t},o_{t}$ 三个函数来控制，因为$ \sigma(x)$（代表sigmoid函数） 的值是介于0到1之间的，刚好用趋近于0时表示流入不能通过gate，趋近于1时表示流入可以通过gate。

$$f_{t}=\sigma({W_{f}X_{t}}+b_{f})$$

$$i_{t}=\sigma({W_{i}X_{t}}+b_{i})$$

$$o_{i}=\sigma({W_{o}X_{t}}+b_{o})$$

当前的状态 $S_{t}=f_{t}S_{t-1}+i_{t}X_{t}$类似与传统RNN$ S_{t}=W_{s}S_{t-1}+W_{x}X_{t}+b_{1}$。  
将LSTM的状态表达式展开后得：

$$S_{t}=\sigma(W_{f}X_{t}+b_{f})S_{t-1}+\sigma(W_{i}X_{t}+b_{i})X_{t}$$

如果加上激活函数， $S_{t}=tanh\left[\sigma(W_{f}X_{t}+b_{f})S_{t-1}+\sigma(W_{i}X_{t}+b_{i})X_{t}\right]$

RNN梯度消失和爆炸的原因这篇文章中传统RNN求偏导的过程包含 $$\prod_{j=k+1}^{t}\frac{\partial{S_{j}}}{\partial{S_{j-1}}}=\prod_{j=k+1}^{t}{tanh{'}W_{s}}$$

对于LSTM同样也包含这样的一项，但是在LSTM中 $\prod_{j=k+1}^{t}\frac{\partial{S_{j}}}{\partial{S_{j-1}}}=\prod_{j=k+1}^{t}{tanh{’}\sigma({W_{f}X_{t}+b_{f}})}$

假设 $Z=tanh{'}(x)\sigma({y})$ ，则 Z 的函数图像如下图所示：  
<img src="./zed.png"></img>  
可以看到该函数值基本上不是0就是1。

传统RNN的求偏导过程：

$$\frac{\partial{L_{3}}}{\partial{W_{s}}}=\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{3}}}{\partial{W_{s}}}+\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{3}}}{\partial{S_{2}}}\frac{\partial{S_{2}}}{\partial{W_{s}}}+\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{3}}}{\partial{S_{2}}}\frac{\partial{S_{2}}}{\partial{S_{1}}}\frac{\partial{S_{1}}}{\partial{W_{s}}}$$

如果在LSTM中上式可能就会变成：

$$\frac{\partial{L_{3}}}{\partial{W_{s}}}=\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{3}}}{\partial{W_{s}}}+\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{2}}}{\partial{W_{s}}}+\frac{\partial{L_{3}}}{\partial{O_{3}}}\frac{\partial{O_{3}}}{\partial{S_{3}}}\frac{\partial{S_{1}}}{\partial{W_{s}}}$$

因为 $\prod_{j=k+1}^{t}\frac{\partial{S_{j}}}{\partial{S_{j-1}}}=\prod_{j=k+1}^{t}{tanh{’}\sigma({W_{f}X_{t}+b_{f}})}\approx0|1$ ，这样就解决了传统RNN中梯度消失的问题。

### Action 1: cnews 中文文本分类：
由清华大学根据新浪新闻RSS订阅频道2005-2011年间的历史数据筛选过滤生成  
训练集 50000  
验证集 5000  
测试集 10000  
词汇（字） 5000  
10个分类，包括：'体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐'  

In [1]:
# 引包
import numpy as np
import torch
from torch import optim
from torch import nn
from model import TextRNN
from cnews_loader import read_vocab, read_category, process_file

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']
x_train= [[1609  659   56 ...    9  311    3]
 [   2  101   16 ... 1168    3   24]
 [ 465  855  521 ...  116  136   85]
 ...
 [  49   18   79 ...  836 1928 1072]
 [ 166  110  714 ...  836 1928 1072]
 [   1   80  551 ...   78  192    3]]


In [2]:
# 设置数据目标
train_file = 'cnews.train.txt'
test_file = 'cnews.test.txt'
val_file = 'cnews.val.txt'
vocab_file = 'cnews.vocab.txt'

In [3]:
def train(Train_Epoch):
    model = TextRNN().cuda()
    # 定义损失函数
    Loss = nn.MultiLabelSoftMarginLoss()
    # 定义优化器
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    best_val_acc = 0
    # 训练
    for epoch in range(Train_Epoch):
        print('epoch=', epoch)
        # 分批训练
        for step, (x_batch, y_batch) in enumerate(train_loader):
            x = x_batch.cuda()
            y = y_batch.cuda()
            out = model(x)
            loss = Loss(out, y)
            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # 计算准确率
            accuracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
        print('train loss=', loss)
        print('train accuracy:', accuracy)
        # 对模型进行验证
        if (epoch+1) % 5 == 0:
            for step, (x_batch, y_batch) in enumerate(val_loader):
                x = x_batch.cuda()
                y = y_batch.cuda()
                out = model(x)
                accuracy = np.mean((torch.argmax(out, 1) == torch.argmax(y, 1)).cpu().numpy())
                if accuracy > best_val_acc:
                    torch.save(model, "model.pkl")
                    best_val_acc = accuracy
                    print('model.pkl saved')
                    print('val accuracy:', accuracy)
    return model

In [4]:
# 获取文本的类别及其对应id的字典
categories, cat_to_id = read_category()
categories

['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐']

In [5]:
# 获取训练文本中所有出现过的字及其所对应的id
words, word_to_id = read_vocab('cnews.vocab.txt')
words

['<PAD>',
 '，',
 '的',
 '。',
 '一',
 '是',
 '在',
 '0',
 '有',
 '不',
 '了',
 '中',
 '1',
 '人',
 '大',
 '、',
 '国',
 '',
 '2',
 '这',
 '上',
 '为',
 '个',
 '“',
 '”',
 '年',
 '学',
 '时',
 '我',
 '地',
 '和',
 '以',
 '到',
 '出',
 '来',
 '会',
 '行',
 '发',
 '：',
 '对',
 '们',
 '要',
 '生',
 '家',
 '他',
 '能',
 '也',
 '业',
 '金',
 '3',
 '成',
 '可',
 '分',
 '多',
 '现',
 '5',
 '就',
 '场',
 '新',
 '后',
 '于',
 '下',
 '日',
 '经',
 '市',
 '前',
 '过',
 '方',
 '得',
 '作',
 '月',
 '最',
 '开',
 '房',
 '》',
 '《',
 '高',
 '9',
 '8',
 '.',
 '而',
 '比',
 '公',
 '4',
 '说',
 ')',
 '将',
 '(',
 '都',
 '资',
 'e',
 '6',
 '基',
 '用',
 '面',
 '产',
 '还',
 '自',
 '者',
 '本',
 '之',
 '美',
 '很',
 '同',
 '',
 '7',
 '部',
 '进',
 '但',
 '主',
 '外',
 '动',
 '机',
 '元',
 '理',
 '加',
 'a',
 '全',
 '与',
 '实',
 '影',
 '好',
 '小',
 '间',
 '其',
 '天',
 '定',
 '表',
 '力',
 '如',
 '次',
 '合',
 '长',
 'o',
 '体',
 '价',
 'i',
 '所',
 '内',
 '子',
 '目',
 '电',
 '-',
 '当',
 '度',
 '品',
 '看',
 '期',
 '关',
 '更',
 'n',
 '等',
 '工',
 '然',
 '斯',
 '重',
 '些',
 '球',
 '此',
 '里',
 '利',
 '相',
 '情',
 '投',
 '点',
 '没',
 '

In [6]:
# 获取训练数据每个字的id和对应标签的one-hot形式
x_train, y_train = process_file(train_file, word_to_id, cat_to_id, 600)
x_train

array([[1609,  659,   56, ...,    9,  311,    3],
       [   2,  101,   16, ..., 1168,    3,   24],
       [ 465,  855,  521, ...,  116,  136,   85],
       ...,
       [  49,   18,   79, ...,  836, 1928, 1072],
       [ 166,  110,  714, ...,  836, 1928, 1072],
       [   1,   80,  551, ...,   78,  192,    3]], dtype=int32)

In [7]:
x_val, y_val = process_file(val_file, word_to_id, cat_to_id, 600)

In [8]:
import torch.utils.data as Data
# 设置GPU
cuda = torch.device('cuda')
x_train, y_train = torch.LongTensor(x_train), torch.Tensor(y_train)
x_val, y_val = torch.LongTensor(x_val), torch.Tensor(y_val)

In [9]:
# 训练参数设置
Batch_Size = 256
Train_Epoch = 40

In [10]:
train_dataset = Data.TensorDataset(x_train, y_train)
train_loader = Data.DataLoader(dataset=train_dataset, batch_size=Batch_Size, shuffle=True)
val_dataset = Data.TensorDataset(x_val, y_val)
val_loader = Data.DataLoader(dataset=val_dataset, batch_size=Batch_Size)

In [11]:
# 模型训练
model = train(Train_Epoch)

epoch= 0


/home/wll/anaconda3/envs/torch/lib/python3.6/site-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


train loss= tensor(0.7221, device='cuda:0', grad_fn=<MeanBackward0>)
train accuracy: 0.275
epoch= 1
train loss= tensor(0.7165, device='cuda:0', grad_fn=<MeanBackward0>)
train accuracy: 0.375
epoch= 2
train loss= tensor(0.7067, device='cuda:0', grad_fn=<MeanBackward0>)
train accuracy: 0.4625
epoch= 3
train loss= tensor(0.7133, device='cuda:0', grad_fn=<MeanBackward0>)
train accuracy: 0.3625
epoch= 4
train loss= tensor(0.7180, device='cuda:0', grad_fn=<MeanBackward0>)
train accuracy: 0.3375
model.pkl saved
val accuracy: 0.90234375
epoch= 5
train loss= tensor(0.7167, device='cuda:0', grad_fn=<MeanBackward0>)
train accuracy: 0.325
epoch= 6
train loss= tensor(0.7098, device='cuda:0', grad_fn=<MeanBackward0>)
train accuracy: 0.4
epoch= 7
train loss= tensor(0.7126, device='cuda:0', grad_fn=<MeanBackward0>)
train accuracy: 0.3625
epoch= 8
train loss= tensor(0.7178, device='cuda:0', grad_fn=<MeanBackward0>)
train accuracy: 0.3375
epoch= 9
train loss= tensor(0.7201, device='cuda:0', grad_fn=<Mea